In [1]:
import os
import time
import pandas as pd
from datetime import datetime, timedelta
import requests

In [2]:
def fetch_candles(pair: str, before: int, after: int, periods: int, save: bool = False) -> pd.DataFrame:
    '''
    exchange: ftx
    pair: fttusd-perpetual-futures
    '''
    url = f"https://api.cryptowat.ch/markets/ftx/{pair}/ohlc"
    query = {'before': before, 'after': after, 'periods': periods}
    candles = requests.get(url, params=query).json()["result"][f"{periods}"]

    t, o, h, l, c, v = [], [], [], [], [], []
    for candle in candles:
        t.append(datetime.fromtimestamp(candle[0]))
        o.append(candle[1])
        h.append(candle[2])
        l.append(candle[3])
        c.append(candle[4])
        v.append(candle[5])

    candles = pd.DataFrame({"Date": t, "Open": o, "High": h, "Low": l, "Close": c, "Volume": v}).set_index("Date")

    if save:
        pth = f"../data/{periods}/{pair}/"
        os.makedirs(pth, exist_ok=True)
        candles.to_csv(pth + f"{candles.index[0].strftime('%Y-%m-%d')}.csv")

    return candles

In [3]:
pair = "btcusd-perpetual-futures"
current_time = datetime.fromtimestamp(int(time.time()))
after = current_time - timedelta(days=300)
print(current_time)
print(after)
before = int(datetime.timestamp(current_time))
after = int(datetime.timestamp(after))
periods = 60 * 60
print(before, after)
print(before, after + 60 * 60 * 24)

candles = fetch_candles(pair, before, after, periods)
print(candles)

2021-07-29 16:05:23
2020-10-02 16:05:23
1627542323 1601622323
1627542323 1601708723
                        Open     High      Low    Close     Volume
Date                                                              
2020-11-21 18:00:00  18694.0  18814.0  18688.0  18814.0   860.9376
2020-11-21 19:00:00  18814.5  18814.5  18576.5  18634.0  1947.8756
2020-11-21 20:00:00  18637.0  18726.0  18592.5  18620.0  1481.4738
2020-11-21 21:00:00  18619.5  18673.5  18511.5  18555.0  1332.0568
2020-11-21 22:00:00  18556.0  18742.5  18301.5  18666.5  4146.2396
...                      ...      ...      ...      ...        ...
2021-07-29 12:00:00  39666.0  39995.0  39626.0  39720.0  1962.1624
2021-07-29 13:00:00  39721.0  39870.0  39517.0  39629.0  1264.2347
2021-07-29 14:00:00  39630.0  40144.0  39500.0  39987.0  2407.5231
2021-07-29 15:00:00  39988.0  40343.0  39722.0  40150.0  7353.8637
2021-07-29 16:00:00  40151.0  40245.0  39938.0  40035.0  1664.7514

[5999 rows x 5 columns]


In [4]:
pairs = ["btcusd", "ethusd"]
url = "https://api.cryptowat.ch/markets/ftx"
markets = requests.get(url).json()["result"]
for market in markets:
    if market["pair"] in pairs:
        print(market)

{'id': 62576, 'exchange': 'ftx', 'pair': 'btcusd', 'active': True, 'route': 'https://api.cryptowat.ch/markets/ftx/btcusd'}
{'id': 62579, 'exchange': 'ftx', 'pair': 'ethusd', 'active': True, 'route': 'https://api.cryptowat.ch/markets/ftx/ethusd'}


In [6]:
start = int(datetime.timestamp(datetime(2021, 1, 1)))
end = int(datetime.timestamp(datetime(2021, 6, 30)))
for pair in pairs:
    fetch_candles(pair, end, start, 60 * 60, save=True)